In [60]:
import os
os.chdir('..')

In [1]:
from torchvision.datasets import EMNIST

In [37]:
digits_train_ds = EMNIST(root="../data", split='byclass', train=True, download=True)

In [45]:
digits_train_ds, next(next(iter(digits_train_ds)))

TypeError: 'tuple' object is not an iterator

In [43]:
digits_train_ds.class_to_idx

{'0 - zero': 0,
 '1 - one': 1,
 '2 - two': 2,
 '3 - three': 3,
 '4 - four': 4,
 '5 - five': 5,
 '6 - six': 6,
 '7 - seven': 7,
 '8 - eight': 8,
 '9 - nine': 9}

In [63]:
import tensorflow_federated as tff
from pathlib import Path
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [11]:
tf_ds = emnist_train.create_tf_dataset_for_client(emnist_train.client_ids[0])

/opt/conda/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [26]:
lens = [len(list(emnist_train.create_tf_dataset_for_client(client_id).as_numpy_iterator())) for client_id in emnist_train.client_ids]

/opt/conda/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [28]:
sum(lens)

341873

In [30]:
lens_test = [len(list(emnist_test.create_tf_dataset_for_client(client_id).as_numpy_iterator())) for client_id in emnist_test.client_ids]

In [31]:
sum(lens_test)

40832

In [67]:
def process_emnist_tff_set(emnist, base_dir):
    os.makedirs(base_dir)
    for client_id in tqdm(emnist.client_ids):
        tf_ds = emnist.create_tf_dataset_for_client(client_id)
        x = [np.expand_dims(x['pixels'], axis=0) for x in tf_ds.as_numpy_iterator()]
        x = np.stack(x)
        y = [np.expand_dims(x['label'], axis=0) for x in tf_ds.as_numpy_iterator()]
        y = np.stack(y)
        np.save(base_dir / client_id, {'x': x, 'y': y})

In [68]:
base_dir = Path('data')
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()
process_emnist_tff_set(emnist_train, base_dir / 'train')
process_emnist_tff_set(emnist_test, base_dir / 'test')

/opt/conda/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
100%|██████████| 3383/3383 [00:46<00:00, 72.86it/s]
